# SQL avancé

Auteur : BLAIN Killian - INFO4 G1

## 1. Chargement et nettoyage des données

In [65]:
import duckdb
import pandas as pd
import sqlalchemy

%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///eco2mix.db

Les données étant contenues dans un fichier [.csv](https://uncloud.univ-nantes.fr/index.php/s/DfiTaK9p33Cb2JJ), on les transfère d'abord dans une table SQL afin de ne pas avoir à écrire ```FROM read_csv_auto('eco2mix-regional-cons-def.csv')``` mais cela évite surtout de ne pas recharger tout le fichier à chaque requête.

In [66]:
%sql CREATE OR REPLACE TABLE eco2mix AS ( SELECT * FROM read_csv_auto('eco2mix-regional-cons-def.csv'));

,Count
0,1980288


Les lignes sont bien rajoutées (environ 2 millions) mais en effectuant une simple requête dessus, on se rend vite compte qu'on a beaucoup de problèmes.

In [67]:
%sql select * from eco2mix LIMIT 100;

,Code INSEE région,Région,Nature,Date,Heure,Date - Heure,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),...,TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%),Column 30
0,28,Normandie,Données définitives,2013-01-01,00:00,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1,27,Bourgogne-Franche-Comté,Données définitives,2013-01-01,00:00,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,32,Hauts-de-France,Données définitives,2013-01-01,00:00,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
3,11,Île-de-France,Données définitives,2013-01-01,00:00,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
4,53,Bretagne,Données définitives,2013-01-01,00:00,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,11,Île-de-France,Données définitives,2013-01-01,03:30,2013-01-01 02:30:00,7667.0,678.0,NaN,16.0,...,None,None,None,None,None,None,None,None,None,None
96,84,Auvergne-Rhône-Alpes,Données définitives,2013-01-01,04:00,2013-01-01 03:00:00,7383.0,252.0,11302.0,221.0,...,None,None,None,None,None,None,None,None,None,None
97,28,Normandie,Données définitives,2013-01-01,04:00,2013-01-01 03:00:00,3069.0,432.0,7334.0,132.0,...,None,None,None,None,None,None,None,None,None,None
98,75,Nouvelle-Aquitaine,Données définitives,2013-01-01,04:00,2013-01-01 03:00:00,4736.0,247.0,5020.0,119.0,...,None,None,None,None,None,None,None,None,None,None


On remarque :
- Les colonnes sont bizarrement nommées, rendant leur utilisation peu pratique
- Certaines colonnes sont inutiles ("Column 30") ou bien dédoublent les informations (les colonnes "Date" et "Heure" sont inutiles étant donné qu'il y a la colonne "Date - Heure")
- Certaines colonnes contiennent beaucoup de NULL

In [68]:
%%sql
SELECT column_name, data_type
FROM duckdb_columns
WHERE table_name = 'eco2mix'

,column_name,data_type
0,Code INSEE région,INTEGER
1,Région,VARCHAR
2,Nature,VARCHAR
3,Date,DATE
4,Heure,VARCHAR
5,Date - Heure,TIMESTAMP
6,Consommation (MW),INTEGER
7,Thermique (MW),INTEGER
8,Nucléaire (MW),INTEGER
9,Eolien (MW),INTEGER


En observant de plus près, on se rend compte que certaines colonnes n'ont pas le bon type. En effet, un pourcentage devrait être une variable numérique, et non pas une chaîne de charactères.

On va donc mettre tout cela au propre.

### Renommage des colonnes

In [69]:
%%sql
ALTER TABLE eco2mix RENAME "Code INSEE région" TO "codeINSEE";
ALTER TABLE eco2mix RENAME "Région" TO "region";
ALTER TABLE eco2mix RENAME "Nature" TO "nature";
ALTER TABLE eco2mix RENAME "Date" TO "date";
ALTER TABLE eco2mix RENAME "Heure" TO "heure";
ALTER TABLE eco2mix RENAME "Date - Heure" TO "dateTime";
ALTER TABLE eco2mix RENAME "Consommation (MW)" TO "consommation";
ALTER TABLE eco2mix RENAME "Thermique (MW)" TO "thermique";
ALTER TABLE eco2mix RENAME "Nucléaire (MW)" TO "nucleaire";
ALTER TABLE eco2mix RENAME "Eolien (MW)" TO "eolien";
ALTER TABLE eco2mix RENAME "Solaire (MW)" TO "solaire";
ALTER TABLE eco2mix RENAME "Hydraulique (MW)" TO "hydraulique";
ALTER TABLE eco2mix RENAME "Pompage (MW)" TO "pompage";
ALTER TABLE eco2mix RENAME "Bioénergies (MW)" TO "bioenergies";
ALTER TABLE eco2mix RENAME "Ech. physiques (MW)" TO "echangesPhysiques";
ALTER TABLE eco2mix RENAME "Stockage batterie" TO "stockageBatterie";
ALTER TABLE eco2mix RENAME "Déstockage batterie" TO "destockageBatterie";
ALTER TABLE eco2mix RENAME "Eolien terrestre" TO "eolienTerrestre";
ALTER TABLE eco2mix RENAME "Eolien offshore" TO "eolienOffshore";
ALTER TABLE eco2mix RENAME "TCO Thermique (%)" TO "tcoThermique";
ALTER TABLE eco2mix RENAME "TCH Thermique (%)" TO "tchThermique";
ALTER TABLE eco2mix RENAME "TCO Nucléaire (%)" TO "tcoNucleaire";
ALTER TABLE eco2mix RENAME "TCH Nucléaire (%)" TO "tchNucleaire";
ALTER TABLE eco2mix RENAME "TCO Eolien (%)" TO "tcoEolien";
ALTER TABLE eco2mix RENAME "TCH Eolien (%)" TO "tchEolien";
ALTER TABLE eco2mix RENAME "TCO Solaire (%)" TO "tcoSolaire";
ALTER TABLE eco2mix RENAME "TCH Solaire (%)" TO "tchSolaire";
ALTER TABLE eco2mix RENAME "TCO Hydraulique (%)" TO "tcoHydraulique";
ALTER TABLE eco2mix RENAME "TCH Hydraulique (%)" TO "tchHydraulique";
ALTER TABLE eco2mix RENAME "TCO Bioénergies (%)" TO "tcoBioenergies";
ALTER TABLE eco2mix RENAME "TCH Bioénergies (%)" TO "tchBioenergies";
ALTER TABLE eco2mix RENAME "Column 30" TO "col30";

""


### Retypage des colonnes

In [70]:
%%sql
ALTER TABLE eco2mix ALTER consommation TYPE DOUBLE;
ALTER TABLE eco2mix ALTER thermique TYPE DOUBLE;
ALTER TABLE eco2mix ALTER nucleaire TYPE DOUBLE;
ALTER TABLE eco2mix ALTER eolien TYPE DOUBLE;
ALTER TABLE eco2mix ALTER solaire TYPE DOUBLE;
ALTER TABLE eco2mix ALTER hydraulique TYPE DOUBLE;
ALTER TABLE eco2mix ALTER pompage TYPE DOUBLE;
ALTER TABLE eco2mix ALTER bioenergies TYPE DOUBLE;
ALTER TABLE eco2mix ALTER echangesPhysiques TYPE DOUBLE;
ALTER TABLE eco2mix ALTER stockageBatterie TYPE DOUBLE;
ALTER TABLE eco2mix ALTER destockageBatterie TYPE DOUBLE;
ALTER TABLE eco2mix ALTER eolienTerrestre TYPE DOUBLE;
ALTER TABLE eco2mix ALTER eolienOffshore TYPE DOUBLE;
ALTER TABLE eco2mix ALTER tcoThermique TYPE DOUBLE;
ALTER TABLE eco2mix ALTER tchThermique TYPE DOUBLE;
ALTER TABLE eco2mix ALTER tcoNucleaire TYPE DOUBLE;
ALTER TABLE eco2mix ALTER tchNucleaire TYPE DOUBLE;
ALTER TABLE eco2mix ALTER tcoEolien TYPE DOUBLE;
ALTER TABLE eco2mix ALTER tchEolien TYPE DOUBLE;
ALTER TABLE eco2mix ALTER tcoSolaire TYPE DOUBLE;
ALTER TABLE eco2mix ALTER tchSolaire TYPE DOUBLE;
ALTER TABLE eco2mix ALTER tcoHydraulique TYPE DOUBLE;
ALTER TABLE eco2mix ALTER tchHydraulique TYPE DOUBLE;
ALTER TABLE eco2mix ALTER tcoBioenergies TYPE DOUBLE;
ALTER TABLE eco2mix ALTER tchBioenergies TYPE DOUBLE;

""


### Suppression des colonnes inutiles

Les colonnes nature et col30 sont inutiles. Les colonnes date et heure ne sont plus utiles car on a la colonne dateTime qui contient la date et l'heure.

In [71]:
%%sql
ALTER TABLE eco2mix DROP nature;
ALTER TABLE eco2mix DROP date;
ALTER TABLE eco2mix DROP heure;
ALTER TABLE eco2mix DROP col30;
DELETE FROM eco2mix WHERE year(dateTime)=2012; --Il y a quelques lignes qui datent de 2012 mais pas assez pour que cela nous intéresse

,Count
0,24


### Traitement des valeurs NULL

On remplace les valeurs NULL par 0 pour éviter d'utiliser IFNULL ou COALESCE à chaque requête où l'on utilise ces colonnes.

In [72]:
%%sql
UPDATE eco2mix SET consommation = 0 WHERE consommation IS NULL;
UPDATE eco2mix SET thermique = 0 WHERE thermique IS NULL;
UPDATE eco2mix SET nucleaire = 0 WHERE nucleaire IS NULL;
UPDATE eco2mix SET eolien = 0 WHERE eolien IS NULL;
UPDATE eco2mix SET solaire = 0 WHERE solaire IS NULL;
UPDATE eco2mix SET hydraulique = 0 WHERE hydraulique IS NULL;
UPDATE eco2mix SET pompage = 0 WHERE pompage IS NULL;
UPDATE eco2mix SET bioenergies = 0 WHERE bioenergies IS NULL;
UPDATE eco2mix SET echangesPhysiques = 0 WHERE echangesPhysiques IS NULL;
UPDATE eco2mix SET stockageBatterie = 0 WHERE stockageBatterie IS NULL;
UPDATE eco2mix SET destockageBatterie = 0 WHERE destockageBatterie IS NULL;
UPDATE eco2mix SET eolienTerrestre = 0 WHERE eolienTerrestre IS NULL;
UPDATE eco2mix SET eolienOffshore = 0 WHERE eolienOffshore IS NULL;
UPDATE eco2mix SET tcoThermique = 0 WHERE tcoThermique IS NULL;
UPDATE eco2mix SET tchThermique = 0 WHERE tchThermique IS NULL;
UPDATE eco2mix SET tcoNucleaire  = 0 WHERE tcoNucleaire IS NULL;
UPDATE eco2mix SET tchNucleaire = 0 WHERE tchNucleaire IS NULL;
UPDATE eco2mix SET tcoEolien = 0 WHERE tcoEolien IS NULL;
UPDATE eco2mix SET tchEolien = 0 WHERE tchEolien IS NULL;
UPDATE eco2mix SET tcoSolaire = 0 WHERE tcoSolaire IS NULL;
UPDATE eco2mix SET tchSolaire = 0 WHERE tchSolaire IS NULL;
UPDATE eco2mix SET tcoHydraulique = 0 WHERE tcoHydraulique IS NULL;
UPDATE eco2mix SET tchHydraulique = 0 WHERE tchHydraulique IS NULL;
UPDATE eco2mix SET tcoBioenergies = 0 WHERE tcoBioenergies IS NULL;
UPDATE eco2mix SET tchBioenergies = 0 WHERE tchBioenergies IS NULL;

,Count
0,1720552


### Consolidation du TCO et ajout d'une colonne

On rajoute la colonne productionTotale puis on calcule le taux de couverture pour chaque type d'énergie.

In [73]:
%%sql
ALTER TABLE eco2mix ADD productionTotale DOUBLE;
UPDATE eco2mix SET productionTotale = thermique+nucleaire+eolien+solaire+hydraulique+pompage+bioenergies+eolienTerrestre+eolienOffshore;

UPDATE eco2mix SET tcoThermique = thermique / productionTotale * 100;
UPDATE eco2mix SET tcoNucleaire = nucleaire / productionTotale * 100;
UPDATE eco2mix SET tcoEolien = (eolien+eolienTerrestre+eolienOffshore) / productionTotale * 100;
UPDATE eco2mix SET tcoSolaire = solaire / productionTotale * 100;
UPDATE eco2mix SET tcoHydraulique = hydraulique / productionTotale * 100;
UPDATE eco2mix SET tcoBioenergies = bioenergies / productionTotale * 100;

,Count
0,1980264


## Exploration

### 1. Groupement et agrégation simples

In [74]:
%%sql
SELECT region AS "Région", month(dateTime) as "Mois", year(dateTime) as "Année",
SUM(productionTotale*24*30/1000) as "Production (GWh)",
SUM(consommation*24*30/1000) as "Consommation (GWh)",
MIN(productionTotale) as "Minimum de production (MW)",
MAX(productionTotale) as "Maximum de production (MW)",
MIN(consommation) as "Minimum de consommation (MW)",
MAX(consommation) as "Maximum de consommation (MW)",
AVG(productionTotale) as "Production moyenne (MW)",
AVG(consommation) as "Consommation moyenne (MW)" FROM eco2mix
GROUP BY Région, Mois, Année;

,Région,Mois,Année,Production (GWh),Consommation (GWh),Minimum de production (MW),Maximum de production (MW),Minimum de consommation (MW),Maximum de consommation (MW),Production moyenne (MW),Consommation moyenne (MW)
0,Provence-Alpes-Côte d'Azur,8,2013,1731265.20,4447215.36,659.0,2605.0,2985.0,5341.0,1615.950941,4151.000000
1,Auvergne-Rhône-Alpes,8,2013,12201183.36,6177246.48,6818.0,15873.0,4101.0,7490.0,11388.500000,5765.799059
2,Grand Est,8,2013,10865642.40,4312416.24,6520.0,12894.0,2783.0,5550.0,10141.915323,4025.179435
3,Centre-Val de Loire,8,2013,6296371.92,1718003.52,3976.0,7780.0,1034.0,2119.0,5876.989919,1603.572581
4,Hauts-de-France,8,2013,4556306.88,4838479.20,3418.0,5497.0,3292.0,5882.0,4252.825269,4516.202957
...,...,...,...,...,...,...,...,...,...,...,...
1351,Grand Est,10,2018,12869384.40,5456350.80,7407.0,16650.0,3443.0,7147.0,12028.361373,5099.774563
1352,Nouvelle-Aquitaine,7,2018,6522667.92,4644128.88,3518.0,7999.0,2976.0,5627.0,6088.213038,4334.797715
1353,Centre-Val de Loire,8,2018,8946499.68,1808494.56,6498.0,10274.0,1171.0,2213.0,8350.600806,1688.036290
1354,Auvergne-Rhône-Alpes,7,2018,12432721.68,7167109.68,6810.0,14936.0,4832.0,8272.0,11604.616263,6689.730511


Ici, on obtient les données de production et de consommation par région, mois et année.

### 2. Pivot

In [75]:
%%sql
SELECT datetrunc('day', datetime) as "Jour",
SUM(CASE WHEN region == 'Auvergne-Rhône-Alpes' THEN consommation END)*24/1000 AS "Consommation Auvergne-Rhône-Alpes (GWh)",
SUM(CASE WHEN region == 'Bourgogne-Franche-Comté' THEN consommation END)*24/1000 AS "Consommation Bourgogne-Franche-Comté (GWh)",
SUM(CASE WHEN region == 'Bretagne' THEN consommation END)*24/1000 AS "Consommation Bretagne (GWh)",
SUM(CASE WHEN region == 'Centre-Val de Loire' THEN consommation END)*24/1000 AS "Consommation Centre-Val de Loire (GWh)",
SUM(CASE WHEN region == 'Grand Est' THEN consommation END)*24/1000 AS "Consommation Grand Est (GWh)",
SUM(CASE WHEN region == 'Hauts-de-France' THEN consommation END)*24/1000 AS "Consommation Hauts-de-France (GWh)",
SUM(CASE WHEN region == 'Normandie' THEN consommation END)*24/1000 AS "Consommation Normandie (GWh)",
SUM(CASE WHEN region == 'Nouvelle-Aquitaine' THEN consommation END)*24/1000 AS "Consommation Nouvelle-Aquitaine (GWh)",
SUM(CASE WHEN region == 'Occitanie' THEN consommation END)*24/1000 AS "Consommation Occitanie (GWh)",
SUM(CASE WHEN region == 'Pays de la Loire' THEN consommation END)*24/1000 AS "Consommation Pays de la Loire (GWh)",
FROM eco2mix
GROUP BY Jour
ORDER BY Jour;

,Jour,Consommation Auvergne-Rhône-Alpes (GWh),Consommation Bourgogne-Franche-Comté (GWh),Consommation Bretagne (GWh),Consommation Centre-Val de Loire (GWh),Consommation Grand Est (GWh),Consommation Hauts-de-France (GWh),Consommation Normandie (GWh),Consommation Nouvelle-Aquitaine (GWh),Consommation Occitanie (GWh),Consommation Pays de la Loire (GWh)
0,2013-01-01,8704.968,2703.000,3004.392,2535.312,5376.264,6235.416,3831.864,5725.944,5266.992,3375.384
1,2013-01-02,10079.568,3198.192,3687.672,3115.056,6481.776,7430.544,4600.368,6967.488,6044.448,4394.472
2,2013-01-03,10787.736,3477.312,3635.856,3080.736,7007.688,7682.016,4562.520,7169.544,6446.472,4346.928
3,2013-01-04,10270.680,3234.000,3538.512,2827.056,6703.488,7504.800,4433.448,6841.752,6041.832,4201.224
4,2013-01-05,9525.792,2964.048,3286.992,2733.504,6221.784,7141.968,4243.416,6642.000,5689.152,3943.008
...,...,...,...,...,...,...,...,...,...,...,...
3433,2022-05-27,6575.832,2058.720,2341.872,1766.064,4688.448,4465.440,2852.064,4242.048,3752.376,2614.632
3434,2022-05-28,6228.768,1913.712,2083.416,1677.744,4435.872,4535.208,2716.224,4054.944,3609.600,2374.536
3435,2022-05-29,6053.880,1839.312,1907.232,1629.168,4186.560,5305.704,2635.752,3895.392,3452.112,2252.736
3436,2022-05-30,7274.832,2342.208,2444.088,2029.656,5070.240,5357.328,3097.200,4540.920,3958.440,2928.696


Ici, on calcule la consommation journalière de chaque région en les rangeant par date, ce qui rend très pratique la recherche d'une date.

### 3. Fenêtre glissante

In [76]:
%%sql
SELECT datetrunc('day', datetime) AS "Date",
(SUM(SUM(consommation)) OVER (
    ORDER BY "Date"
    RANGE INTERVAL 30 DAYS PRECEDING)) AS "Consommation du mois écoulé (GWh)"
FROM eco2mix
GROUP BY "Date"
ORDER BY "Date"

,Date,Consommation du mois écoulé (GWh)
0,2013-01-01,2615508.0
1,2013-01-02,5722720.0
2,2013-01-03,8919355.0
3,2013-01-04,11986335.0
4,2013-01-05,14870849.0
...,...,...
3433,2022-05-27,65489854.0
3434,2022-05-28,65029925.0
3435,2022-05-29,64566982.0
3436,2022-05-30,64455743.0


Ici, on obtient la consommation des 30 derniers jours.

### 4. Variation

In [77]:
%%sql
WITH cte(date, consommation_journaliere) AS (
    SELECT datetrunc('day', datetime) AS "Date", SUM(consommation)*24/1000 as "Consommation journalière (GWh)" FROM eco2mix
    GROUP BY "Date"
    ORDER BY "Date"
)
SELECT date, consommation_journaliere, consommation_journaliere - LEAD(consommation_journaliere, -1, consommation_journaliere) OVER (ORDER BY date) as "Différence"
FROM cte
ORDER BY "Différence" DESC
LIMIT 20;

,date,consommation_journaliere,Différence
0,2018-10-29,73412.880,15014.208
1,2015-01-05,82475.280,14557.296
2,2013-03-11,76462.584,14391.048
3,2021-02-08,81960.168,13792.536
4,2018-11-19,79243.608,13787.040
5,2018-02-26,94735.344,13681.032
6,2017-11-13,74351.544,13670.400
7,2017-11-06,70756.680,13535.184
8,2013-01-14,88060.128,13530.624
9,2016-11-07,75728.184,13500.960


La CTE renvoie tout simplement la consommation par jour. Le LEAD(x, -1, x) reviendrait à faire un LAG tout simplement. On finit donc avec les jours qui ont beaucoup plus consommé que la veille.

### 5. Quantité cumulée

In [78]:
%%sql
WITH cte1(date, consommation, prodRenouvelable) AS (
    SELECT datetrunc('day', datetime), SUM(consommation), SUM(eolien+solaire+hydraulique+pompage+bioenergies+eolienTerrestre+eolienOffshore)
    FROM eco2mix
    GROUP BY datetrunc('day', datetime)
),
cte2(date, consommationCumulee, prodRenouvelable) AS (
    SELECT date,
    SUM(consommation) OVER (PARTITION BY year(date) ORDER BY date),
    SUM(prodRenouvelable) OVER (PARTITION BY year(date)),
  FROM
    cte1
)
SELECT MIN(date) as "Jour de dépassement" FROM cte2
WHERE consommationCumulee > prodRenouvelable
GROUP BY year(date);

,Jour de dépassement
0,2013-02-24
1,2014-02-26
2,2015-02-21
3,2016-02-29
4,2017-02-20
5,2018-03-06
6,2019-03-08
7,2020-03-20
8,2021-03-23
9,2022-02-01


La première CTE renvoie juste la consommation et la production des énergies renouvelables. La deuxième CTE permet de séparer les données par année. Finalement, on renvoie simplement la plus petite date pour laquelle la consommation a dépassé la production d'énergies renouvelables. On se rend donc compte que l'on dépasse le jour de dépassement en février ou en mars, alors que le jour du dépassement mondial était le 28 juillet en 2022.

### 6. Calcul de point fixe

### 7. Construction du cube

In [79]:
%%sql
SELECT datetrunc('day', datetime) AS "Jour", datetrunc('month', datetime) AS "Mois", datetrunc('year', datetime) AS "Année", region, SUM(consommation) AS "Conso" FROM eco2mix
GROUP BY CUBE("Jour", "Mois", "Année", region);

,Jour,Mois,Année,region,Conso
0,None,None,None,None,8.902882e+09
1,2018-11-04,None,None,None,2.541304e+06
2,2018-11-05,None,None,None,2.748388e+06
3,2018-11-06,None,None,None,2.744261e+06
4,2018-11-07,None,None,None,2.829281e+06
...,...,...,...,...,...
181852,None,None,None,Bretagne,4.222913e+08
181853,None,None,None,Bourgogne-Franche-Comté,4.025595e+08
181854,None,None,None,Grand Est,8.577425e+08
181855,None,None,None,Hauts-de-France,9.522873e+08


Ici, on a simplement le cube par rapport au jour, mois, année et région, ne sachant pas comment faire pour les zones.